In [13]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
from dotenv import load_dotenv 
load_dotenv()
import yaml
import os
# Core CrewAI components
from crewai import Agent, Task, Crew, Process
from pydantic import BaseModel
from typing import List

# create the github_tools.py
# src/portfolio_assistant/tools/github_tools.py
import requests
import os
# This is the correct way to make a function usable as a tool in CrewAI
from crewai.tools import tool # CrewAI uses LangChain's tool definition standard
from crewai_tools import PDFSearchTool

import agentops
agentops.init(api_key=os.getenv("AGENTOPS_API_KEY"))

In [2]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [3]:
# Define file paths for YAML configurations
files = {
    'agents': r'src\portfolio_assistant\config\agents.yaml',
    'tasks': r'src\portfolio_assistant\config\tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

In [7]:
class WorkExperience(BaseModel):
    job_title: str
    company: str
    start_date: str
    end_date: str
    description: str


class Education(BaseModel):
    degree: str
    institution: str
    graduation_year: str


class Project(BaseModel):
    name: str
    description: str
    technologies: List[str]


class Certification(BaseModel):
    name: str
    issuing_organization: str


class CVInfo(BaseModel):
    work_experience: List[WorkExperience]
    education: List[Education]
    skills: List[str]
    projects: List[Project]
    certifications: List[Certification]


class CVReference(BaseModel):
    section: str  # Example: "work_experience", "skills", "education"
    details: str  # Summary of relevant data from `CVInfo`


class VisitorResponse(BaseModel):
    question: str
    answer: str
    references: List[CVReference]  # Where the answer is derived from

In [8]:
DEFAULT_GITHUB_USERNAME = "achafi"
@tool("list all the repos")
def list_github_repos(github_username: str = DEFAULT_GITHUB_USERNAME) -> str:
    """
    Fetches and lists public repositories for a specified GitHub username.
    Returns a formatted string listing repository names and their descriptions.
    If no username is provided, it defaults to 'achafi'.
    """
    if not github_username:
        github_username = DEFAULT_GITHUB_USERNAME
        # print(f"No username provided, defaulting to {DEFAULT_GITHUB_USERNAME}")

    api_url = f"https://api.github.com/users/{github_username}/repos"
    headers = {"Accept": "application/vnd.github.v3+json"}
    repo_list_str = f"Public GitHub Repositories for {github_username}:\n"

    # print(f"Attempting to fetch repos for {github_username} from {api_url}...")

    try:
        response = requests.get(api_url, headers=headers, params={'type': 'owner', 'sort': 'updated', 'per_page': 100})
        response.raise_for_status()
        repositories = response.json()

        if not repositories:
            # print(f"No public repositories found for user {github_username}.")
            return f"No public repositories found for user {github_username}."

        # print(f"Found {len(repositories)} repositories.")
        repo_details = []
        for repo in repositories:
            name = repo.get("name", "N/A")
            description = repo.get("description", "No description provided.")
            description_text = description if description else "No description provided."
            repo_details.append(f"- {name}: {description_text}")

        if not repo_details:
             return f"No public repositories found or accessible for {github_username}."

        return repo_list_str + "\n".join(repo_details)

    except requests.exceptions.RequestException as e:
        error_message = f"Error fetching GitHub repositories for {github_username}: {e}"
        print(error_message)
        if isinstance(e, requests.exceptions.HTTPError) and e.response.status_code == 404:
            return f"Error: GitHub user '{github_username}' not found."
        return error_message
    except Exception as e:
        error_message = f"An unexpected error occurred: {e}"
        print(error_message)
        return error_message



In [10]:

# The config here is necessary for THIS SPECIFIC TOOL to function internally
pdf_search_tool = PDFSearchTool(
    pdf="knowledge/CV_Assia_Chafi_2025.pdf", # Ensure path is correct
    config=dict(
        llm=dict(
            provider="openai", # Using OpenAI models via CrewAI's abstractions
            config=dict(model="gpt-3.5-turbo"),
        ),
        embedder=dict(
            provider="openai", # Using OpenAI embeddings via CrewAI's abstractions
            config=dict(model="text-embedding-ada-002"),
        ),
    )
)
# Provide a description for the agent to understand when to use the tool
pdf_search_tool.description = "Search Assia Chafi's CV PDF with a text query to find relevant information about experience, skills, projects, or education."


In [11]:
# # Now create your agents
# from crewai import Agent, Task
# researcher = Agent(config=agents_config["resume_assistant"], verbose=True)
# reporting_analyst = Agent(config=agents_config["reporting_analyst"], verbose=True)

# research_task = Task(
#     description=tasks_config["extract_cv_info"]["description"],
#     expected_output=tasks_config["extract_cv_info"]['expected_output'],
#     output_pydantic=CVInfo,
#     tools=[pdf_search_tool],
#     agent = researcher
# )

# reporting_task = Task(
#     description=tasks_config["reporting_task"]['description'],
#     expected_output=tasks_config["reporting_task"]['expected_output'],
#     output_pydantic=VisitorResponse,
#     output_file="report.md",
#     agent=reporting_analyst
# )

In [14]:

# --- Create Agents using CrewAI's Agent class ---
resume_retriever = Agent(
    config=agents_config["resume_assistant"], # Load from YAML
    tools=[pdf_search_tool],                  # Assign CrewAI/LangChain tool
    verbose=True
    # CrewAI will automatically use the default LLM (usually configured via env vars)
    # unless you explicitly pass an llm=... object here.
)

github_lister = Agent(
    config=agents_config["github_project_lister"], # Load from YAML
    tools=[list_github_repos],                 # Assign your custom tool
    verbose=True
)

reporting_analyst = Agent(
    config=agents_config["reporting_analyst"],      # Load from YAML
    verbose=True,
    allow_delegation=False
)

# --- Create Tasks using CrewAI's Task class ---
search_cv_task = Task(
    config=tasks_config["extract_cv_info"],      # Load from YAML
    agent=resume_retriever                      # Assign Agent
)

list_github_projects_task = Task(
    config=tasks_config["list_github_projects"], # Load from YAML
    agent=github_lister                           # Assign Agent
)

reporting_task = Task(
    config=tasks_config["reporting_task"],      # Load from YAML
    agent=reporting_analyst,                    # Assign Agent
    context=[search_cv_task, list_github_projects_task], # CrewAI's context passing
    output_pydantic=VisitorResponse             # CrewAI's Pydantic output feature
)

# --- Create the Crew using CrewAI's Crew class ---
portfolio_crew = Crew(
    agents=[resume_retriever, github_lister, reporting_analyst], # List agents
    tasks=[search_cv_task, list_github_projects_task, reporting_task], # List tasks
    process=Process.sequential,                 # CrewAI process mode
    verbose=True
)

In [79]:
# The given Python dictionary
inputs = {
  'question': "what is the latest project did you work on?",
}

# Run the crew
result = portfolio_crew.kickoff(
  inputs=inputs
)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 51d821d9-8cee-44dc-819e-bcc8c7a92af9                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 1c2a257d-c4fa-45d4-8c63-de8df40fa65c
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 1c2a257d-c4fa-45d4-8c63-de8df40fa65c
       Status: Executing Task...
    └── 🤖 Agent: Resume assistant
        
            Status: In Progress

# Agent: Resume assistant
## Task: Review the resume CV_Assia_Chafi_2025.pdf and extract key details about experience, skills, projects, and achievements.



🚀 Crew: crew
└── 📋 Task: 1c2a257d-c4fa-45d4-8c63-de8df40fa65c
       Status: Executing Task...
    └── 🤖 Agent: Resume assistant
        
            Status: In Progress
        └── 🧠 Thinking...

...



# Agent: Reporting Analyst
## Final Answer: 
{
  "question": "What is the latest project you worked on?",
  "answer": "My most recent project was at PA ATS, where I worked as a Data Science Engineer.  While the provided resume doesn't detail specific project names during this time, my role involved effectively communicating technical information to both technical and non-technical stakeholders.  My previous role at Yazzoom involved a number of projects, including forecasting glass fragmentation quality with 98% accuracy using machine learning and anomaly detection techniques, and developing a PowerBI dashboard to visualize plant energy consumption.  Before that, I worked on a range of projects at AGC Automotive, focused on building image processing algorithms, machine learning systems for malicious activity detection, and Android applications, including a banking chatbot.",
  "references": [
    {
      "section": "Work Experience",
      "details": "PA ATS (01/2022 - Present):  Effe

🚀 Crew: crew
├── 📋 Task: 1c2a257d-c4fa-45d4-8c63-de8df40fa65c
│      Assigned to: Resume assistant
│   
│      Status: ✅ Completed
│   └── 🤖 Agent: Resume assistant
│       
│           Status: ✅ Completed
│       └── 🔧 Used Search a PDF's content (22)
├── 📋 Task: ef19954e-ca2b-4431-9bfb-2db3d4bc93f1
│      Assigned to: GitHub Repository Scout
│   
│      Status: ✅ Completed
│   └── 🤖 Agent: GitHub Repository Scout
│       
│           Status: ✅ Completed
│       └── 🔧 Used list all the repos (4)
└── 📋 Task: 6d61f1c1-aff8-4593-88e6-9afaa50e15df
       Status: Executing Task...
    └── 🤖 Agent: Reporting Analyst
        
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 1c2a257d-c4fa-45d4-8c63-de8df40fa65c
│      Assigned to: Resume assistant
│   
│      Status: ✅ Completed
│   └── 🤖 Agent: Resume assistant
│       
│           Status: ✅ Completed
│       └── 🔧 Used Search a PDF's content (22)
├── 📋 Task: ef19954e-ca2b-4431-9bfb-2db3d4bc93f1
│      Assigned to: GitHub Repository Scout
│   
│      Status: ✅ Completed
│   └── 🤖 Agent: GitHub Repository Scout
│       
│           Status: ✅ Completed
│       └── 🔧 Used list all the repos (4)
└── 📋 Task: 6d61f1c1-aff8-4593-88e6-9afaa50e15df
       Assigned to: Reporting Analyst
    
       Status: ✅ Completed
    └── 🤖 Agent: Reporting Analyst
        
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 6d61f1c1-aff8-4593-88e6-9afaa50e15df                                                                     │
│  Agent: Reporting Analyst                                                                                       │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 51d821d9-8cee-44dc-819e-bcc8c7a92af9                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [80]:
result['answer']

"My most recent project was at PA ATS, where I worked as a Data Science Engineer.  While the provided resume doesn't detail specific project names during this time, my role involved effectively communicating technical information to both technical and non-technical stakeholders.  My previous role at Yazzoom involved a number of projects, including forecasting glass fragmentation quality with 98% accuracy using machine learning and anomaly detection techniques, and developing a PowerBI dashboard to visualize plant energy consumption.  Before that, I worked on a range of projects at AGC Automotive, focused on building image processing algorithms, machine learning systems for malicious activity detection, and Android applications, including a banking chatbot."